In [1]:
# ループ学習iterationで強かったパーティを収集する

In [19]:
from pokeai.ai.party_db import col_party, col_rate, pack_obj, unpack_obj
from bson import ObjectId
import numpy as np
from pokeai.util import json_load, pickle_load, ROOT_DIR, DATASET_DIR, json_dump

In [3]:
file_base_dir = r"D:\dev\pokeai\pokeai\experiment\gmm\rl\rl_loop_200815"

In [4]:
iter_ids = json_load(file_base_dir+"/iter_ids.json")

In [5]:
iter_ids

[{'iternum': 0,
  'trainer_id': '5f37a6829949403bf6af940b',
  'train_party_tags': ['rl_loop_200815_1_allonce_0', 'rl_loop_200815_1_hc_0'],
  'gen_party_tag': 'rl_loop_200815_1_hc_1',
  'rate_id': '5f478d7f60f6bad00ef43ff8'},
 {'iternum': 1,
  'trainer_id': '5f37a6829949403bf6af940c',
  'train_party_tags': ['rl_loop_200815_1_allonce_1', 'rl_loop_200815_1_hc_1'],
  'gen_party_tag': 'rl_loop_200815_1_hc_2',
  'rate_id': '5f478d7f60f6bad00ef43ff9'},
 {'iternum': 2,
  'trainer_id': '5f37a6829949403bf6af940d',
  'train_party_tags': ['rl_loop_200815_1_allonce_2', 'rl_loop_200815_1_hc_2'],
  'gen_party_tag': 'rl_loop_200815_1_hc_3',
  'rate_id': '5f478d7f60f6bad00ef43ffa'},
 {'iternum': 3,
  'trainer_id': '5f37a6829949403bf6af940e',
  'train_party_tags': ['rl_loop_200815_1_allonce_3', 'rl_loop_200815_1_hc_3'],
  'gen_party_tag': 'rl_loop_200815_1_hc_4',
  'rate_id': '5f478d7f60f6bad00ef43ffb'},
 {'iternum': 4,
  'trainer_id': '5f37a6829949403bf6af940f',
  'train_party_tags': ['rl_loop_200815_1

In [22]:
def get_top_rated_players(rate_id_str, top=100):
    rates = col_rate.find_one({"_id": ObjectId(rate_id_str)})["rates"]
    rate_tuples = [(rate, player_id) for player_id, rate in rates.items()]
    rate_tuples.sort()
    return [player_id for rate, player_id in rate_tuples[-top:]]

In [23]:
all_top_player_ids = []
for iter_id in iter_ids:
    all_top_player_ids.extend(get_top_rated_players(iter_id["rate_id"]))

In [24]:
all_top_player_ids

['...']

In [25]:
len(all_top_player_ids)

1000

In [26]:
json_dump({"player_ids": all_top_player_ids}, file_base_dir+"/top_player_ids.json")

各iterationの代表１００パーティずつ、合計１０００パーティをレーティング戦
```
python -m pokeai.ai.generic_move_model.rl_rating_battle "" "" --player_ids_file D:\dev\pokeai\pokeai\experiment\gmm\rl\rl_loop_200815\top_player_ids.json
# rate_id 5f48dfb7a97c3f95899b0142
```

In [6]:
def load_parties(player_ids):
    party_ids = {player_id.split('+')[1] for player_id in player_ids}
    return {party_id:col_party.find_one({"_id":ObjectId(party_id)})["party"] for party_id in party_ids}

In [7]:
rates = col_rate.find_one({"_id": ObjectId(iter_ids[0]["rate_id"])})["rates"]

In [12]:
parties = load_parties(rates.keys())

In [8]:
name2jp = json_load(DATASET_DIR.joinpath("name2jp.json"))["name2jp"]
def print_party(p):
    for poke in p:
        print(",".join([name2jp[poke["species"]],str(poke["level"]), name2jp.get(poke["item"],""), *[name2jp[move] for move in poke["moves"]]]))